In [1]:
import numpy as np
import random

In [3]:
# # get indices of all nonzero positions
# nonzero_pos = np.nonzero(m)
# print(nonzero_pos)
# num_nonzero = len(nonzero_pos[0])



In [4]:
# img_rows = m.shape[0]
# img_cols = m.shape[1]

# # select anchor example
# # select any segment position at random
# rand_anchor_idx = np.random.randint(num_nonzero)
# rand_anchor_idx = 3 # temp
# i = nonzero_pos[0][rand_anchor_idx]
# j = nonzero_pos[1][rand_anchor_idx]
# anchor = [i,j]
# # print(f'i,j:{i,j}')

# # select hard negative example
# hard_negative = None
# radius=1
# while hard_negative is None:
#     # find closest background pixel to anchor to be the hard negative
#     # option 1 - check for background pixels using increasing radius approach
#     square_length = 3 + 2 * (radius - 1)
# #     print(f'radius:{radius}')
# #     print(f'square_length:{square_length}')

#     # get upper left corner of pixel at given radius from (i,j) even if illegal position
#     ul_i = i - radius
#     ul_j = j - radius
# #     print(f'ul_i, ul_j:{ul_i, ul_j}')

#     # calc all positions in radius-length square starting from that upper left position
#     # ignore any positions that are segment pixels or illegal positions
#     candidate_pos = [] 
#     for a in range(square_length):
#         for b in range(square_length):
#             if a == 0 or a == square_length-1:
#                 pos = [ul_i+a,ul_j+b]
#     #             print(f'pos:{pos}')
#                 c1 = pos[0] >= 0 and pos[0] < img_rows
#                 c2 = pos[1] >= 0 and pos[1] < img_cols

#                 if c1 and c2:

#                     # check if position is a segment pixel
#                     # get all segment row positions that match pos[0]
#                     c3 = True
#                     matches_row_idx = np.where(nonzero_pos[0] == pos[0])[0]
#     #                 print(matches_row_idx)

#                     for each in matches_row_idx:
#     #                     print(f'nonzero_pos[1][each]:{nonzero_pos[1][each]}')
#                         if nonzero_pos[1][each] == pos[1]:
#     #                         print(f'segment position:{pos}')
#                             c3 = False


#                     if c3:
#                         candidate_pos.append(pos)
#             else:
#                 if b == 0 or b == square_length-1: 
#                     pos = [ul_i+a,ul_j+b]
#         #             print(f'pos:{pos}')
#                     c1 = pos[0] >= 0 and pos[0] < img_rows
#                     c2 = pos[1] >= 0 and pos[1] < img_cols

#                     if c1 and c2:

#                         # check if position is a segment pixel
#                         # get all segment row positions that match pos[0]
#                         c3 = True
#                         matches_row_idx = np.where(nonzero_pos[0] == pos[0])[0]
#         #                 print(matches_row_idx)

#                         for each in matches_row_idx:
#         #                     print(f'nonzero_pos[1][each]:{nonzero_pos[1][each]}')
#                             if nonzero_pos[1][each] == pos[1]:
#         #                         print(f'segment position:{pos}')
#                                 c3 = False


#                         if c3:
#                             candidate_pos.append(pos)
#     if len(candidate_pos) > 0:
#         # pick any candidate position as a hard negative because it is a closest background pixel to the anchor
#         hard_negative = random.choice(candidate_pos)
#     else:
#         radius += 1
        

# # select positive example
# # choose any segment pixel except anchor as positive
# rand_positive_idx=np.random.randint(num_nonzero)
# while rand_positive_idx == rand_anchor_idx:
#     rand_positive_idx=np.random.randint(num_nonzero)
    
# i = nonzero_pos[0][rand_positive_idx]
# j = nonzero_pos[1][rand_positive_idx]
# positive = [i,j]


# print(f'anchor:{anchor}')
# print(f'positive:{positive}')
# print(f'hard_negative:{hard_negative}')

In [37]:
def get_new_anchor(nonzero_pos, anchors):

    num_nonzero = len(nonzero_pos[0])
    
    # throw error if there are no possible anchor positions left
    if len(anchors) == num_nonzero:
        raise ValueError('All possible anchor positions have been already selected.')
    
    # select any segment position at random
    rand_nonzero_idx = np.random.randint(num_nonzero)
    i = nonzero_pos[0][rand_nonzero_idx]
    j = nonzero_pos[1][rand_nonzero_idx]
    anchor = [i,j]
    
    # ensure anchor has not already been selected
    while anchor in anchors:
        rand_nonzero_idx = np.random.randint(num_nonzero)
        i = nonzero_pos[0][rand_nonzero_idx]
        j = nonzero_pos[1][rand_nonzero_idx]
        anchor = [i,j]
    
    return anchor, rand_nonzero_idx

In [38]:
def position_is_in_image(pos, img_rows, img_cols):
    c1 = pos[0] >= 0 and pos[0] < img_rows
    c2 = pos[1] >= 0 and pos[1] < img_cols

    position_is_in_image = c1 and c2
    return position_is_in_image

In [39]:
def position_is_background(pos, nonzero_pos):
    
    position_is_background = True
    
    # get all segment row positions that match pos[0]
    matches_row_idx = np.where(nonzero_pos[0] == pos[0])[0] # extract array from results

    # check all row position matches to see if matching column position exists
    for each in matches_row_idx:
        if nonzero_pos[1][each] == pos[1]:
            position_is_background = False
        
    return position_is_background

In [40]:
def get_radius_positions(anchor, radius):
    # get (i,j) position of upper left corner at given radius from anchor even if invalid
    ul_i = anchor[0] - radius
    ul_j = anchor[1] - radius

    # set the length of the square formed at the current radius away from anchor 
    square_length = 3 + 2 * (radius - 1)

    # calc all valid positions that are radius pixels away from the anchor
    # ignore any positions that are segment pixels or invalid positions
    radius_positions = [] 
    for i_offset in range(square_length):
            if i_offset == 0 or i_offset == square_length-1:  # first row or last row

                # every position in first or last row is radius pixels from anchor
                for j_offset in range(square_length):
                    pos = [ul_i+i_offset,ul_j+j_offset]
                    radius_positions.append(pos)

            else: # any row except first or last

                # only first and last columns are radius pixels from anchor
                for j_offset in [0,square_length-1]:
                    pos = [ul_i+i_offset,ul_j+j_offset]
                    radius_positions.append(pos)
                    
    return radius_positions

In [41]:
def get_new_negative(nonzero_pos, anchor, img_rows, img_cols):
    # check for background pixels using increasing radius approach
    # closest background pixel to anchor to be the hard negative
    
    negative = None
    radius=1 # start search 1 pixel away from anchor
    
    while negative is None:

        radius_positions = get_radius_positions(anchor, radius)
#         print(radius_positions)
                        
        # check all radius positions for valid candidates
        candidates = []
        for pos in radius_positions:
            is_valid = position_is_in_image(pos, img_rows, img_cols)
            if is_valid:
                is_background = position_is_background(pos, nonzero_pos)
                if is_background:
                    candidates.append(pos)
                        
        # select candidate at random if any exist
        if len(candidates) > 0:
            negative = random.choice(candidates)
        else:
            # no candidates exist at that radius
            # increment radius and search again
            radius += 1
            
            # sanity check
            if radius > img_rows:
                raise ValueError('Target does not have any valid negative samples.')
                        
    return negative        

In [42]:
def get_new_positive(nonzero_pos, anchor_idx):
    num_nonzero = len(nonzero_pos[0])
    
    if num_nonzero < 2:
        raise ValueError('Target does not have enough segment pixels to choose a positive sample.')
    
    # choose any segment pixel except anchor as positive
    rand_positive_idx=np.random.randint(num_nonzero)
    while rand_positive_idx == anchor_idx:
        rand_positive_idx=np.random.randint(num_nonzero)

    # convert index to an (i,j) pixel position
    i = nonzero_pos[0][rand_positive_idx]
    j = nonzero_pos[1][rand_positive_idx]
    positive = [i,j]
    return positive

In [43]:
def get_triplet_indices(target, n):
    
    # for storing n triplets
    anchors = []
    positives = []
    negatives = []
    
    # extract dimensions of target image
    img_rows = target.shape[0]
    img_cols = target.shape[1]
    
    # get indices of all nonzero positions (segment pixels)
    nonzero_pos = np.nonzero(target)
    
    for triplet in range(n):
        # choose as anchor a segment pixel which has not yet been chosen to be an anchor
        curr_anchor, curr_anchor_idx = get_new_anchor(nonzero_pos, anchors)
        
        # calculate closest background pixel to be hard negative for this anchor
        curr_negative = get_new_negative(nonzero_pos, curr_anchor, img_rows, img_cols)
        
        # choose as positive a segment pixel which is not the current anchor
        curr_positive = get_new_positive(nonzero_pos, curr_anchor_idx)
        
        # save triplet
        anchors.append(curr_anchor)
        negatives.append(curr_negative)
        positives.append(curr_positive)
        
    return anchors, negatives, positives
    

In [63]:
target = np.array([[0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1.]])
print(target.shape)
target

(5, 5)


array([[0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1.]])

In [64]:
# target = np.zeros((5,5))
# print(target.shape)
# target

In [65]:
# target = np.ones((5,5))
# print(target.shape)
# target

In [66]:
# target = np.zeros((5,5))
# target[0][0]=1
# print(target.shape)
# target

In [73]:
n=2
get_triplet_indices(target, n)

([[4, 1], [4, 4]], [[3, 1], [4, 3]], [[0, 2], [0, 4]])